In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 80)

import random

import sys
sys.path.append("../")

from src.object import *
from src.create import *

from config.formation import *

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
df_name = pd.read_csv("../data/csv/NationalNames.csv")
df_name_list = list(df_name[df_name.Gender=='M'].Name.sample(500))

df_place = pd.read_csv("../data/csv/results.csv")
df_place = df_place.city.unique()

In [3]:
class Team:
    def __init__(self, name, formation):
        self.name = name
        self.formation = formation
        self.relegation = 0
        self.promotion = 0
        self.result = pd.DataFrame(columns=['win', 'lose', 'row', '得点', '失点', '得失点差', 'Points', '順位', 'リーグ名'])
        self.competition_result = {}
        self.affilation_players = None
        self.league_name = None
    
    def set_affilation_players_rate(self):
        output = pd.DataFrame(columns=ALL_POSITON)
        output_gk = pd.DataFrame(columns=["GK"])

        for i, p in enumerate(self.affilation_players):
            if p.main_position!="GK":
                df = p.position_all_rate
                buff = pd.DataFrame(df, index=[i])
                buff.loc[i, "injury"] = int(p.injury)
                output = pd.concat([output, buff])
            else:
                output_gk = pd.concat([output_gk, pd.DataFrame([p.main_rate], columns=["GK"], index=[i])])
        output["partification"] = 0
        self.affilation_players_all_rate = output
        self.affilation_players_gk_rate = output_gk
    
    def set_onfield_players(self):
        self.formation.set_players_position()
        for fp in self.formation.formation_priority:
            select_num = self.formation.formation_num[fp]
            df = self.affilation_players_all_rate[((self.affilation_players_all_rate.partification==0)&(self.affilation_players_all_rate.injury<1))]
            select_index = df.sort_values(fp, ascending=False).index.values[:select_num]
            self.affilation_players_all_rate.loc[select_index, "partification"] = 1
            
            for index in select_index:
                self.affilation_players[index].partification = 1
                self.affilation_players[index].partification_position = fp
                self.formation.players[fp].append(self.affilation_players[index])
        
        gk_index = self.affilation_players_gk_rate.sort_values("GK", ascending=False).index.values[0]
        self.formation.players["GK"].append(self.affilation_players[gk_index])

        for fps in self.formation.players.values():
            if fps[0].main_position!="GK":
                self.formation.players_flat.extend(fps)

In [4]:
league_team = []

min_rate = np.random.randint(70, 78)
max_rate = min_rate + 15
players = random_create_players(27, 3, min_rate, max_rate, df_name_list)
formation = random_create_formation()
A = Team(name=df_place[0], 
         formation=formation)

A.affilation_players = players

A.set_affilation_players_rate()
A.set_onfield_players()

In [7]:
A.affilation_players[2].injury_possibility

5.5198698734802015e-05

11